# Uploading it to a Data Warehousing using a script

**Data Dictionary**: Sheet 1 (Cleaned), Sheet 2 (Uncleaned)
 https://docs.google.com/spreadsheets/d/1IdQ7iA6eHaSGUW5JOBv3bBK6NkYCvbhffg5YaxwN5Bs/edit?usp=sharing

# Grant Required Permissions to your Google Cloud Service Account to Create a BigQuery Data Set

**1. Grant Permissions:**
- Go to the Google Cloud Console.
- Navigate to the IAM & Admin > IAM page.
- Locate the user account associated with the credentials you are using.
- Click "ADD IAM CONDITION"
- Under the "Role" field, select the "BigQuery Data Editor"
- Click "SAVE" to grant permissions.

**Note: The BigQuery Data Editor role allows your service account access to edit all the contents of datasets. This step is important for loading your dataset from Google Cloud to the BigQuery Data Warehouse.**

# Install the google-cloud-storage library

In [4]:
%%capture
pip install google-cloud-storage

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


# Install the google-cloud-bigquery library

In [3]:
%%capture
pip install google-cloud-bigquery

# Install upgraded bigquery library

In [2]:
%%capture
pip install --upgrade google-cloud-bigquery

# Install pandas gcsfs

In [1]:
%%capture
pip install pandas gcsfs

# Install pyarrow library

**NOTE: Once pyarrow is installed, you should be able to use the load_table_from_dataframe function without encountering the ValueError from the "Load Data into BigQuery Tables" Cell.**

**After installing pyarrow, you might need to restart your Python environment or Jupyter Notebook kernel before running the script again to ensure that the changes take effect.**

In [ ]:
%%capture
pip install pyarrow

# Import the Python 'os' module

In [ ]:
# CREATE A GOOGLE ACCESSKEY
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Omakasi/GOOGLE_CLOUD_ACCESSKEY.json'

# Install pyarrow library

**NOTE: Once pyarrow is installed, you should be able to use the load_table_from_dataframe function without encountering the ValueError from the "Load Data into BigQuery Tables" Cell.**

**After installing pyarrow, you might need to restart your Python environment or Jupyter Notebook kernel before running the script again to ensure that the changes take effect.**

In [ ]:
pip install pyarrow

# Import the Python 'os' module

In [ ]:
from google.cloud import bigquery

# Create BigQuery Dataset

In [ ]:
from google.cloud import bigquery

# Creating a function for creating a BigQuery dataset with your file stored in your Google Cloud
def create_bigquery_dataset(project_id, dataset_name):
    bigquery_client = bigquery.Client(project=project_id)
    dataset_id = f"{project_id}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset)
    print(f"Dataset {dataset_id} created.")


project_id = 'centering-seer-407304'
dataset_name = '2ndjsoncmepersonalproject'
create_bigquery_dataset(project_id, dataset_name)

Dataset centering-seer-407304.2ndjsoncmepersonalproject created.


# Create Tables in BigQuery

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os

# Getting the path to the service account key file from the environment variable
service_account_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

# Setting your Google Cloud credentials using the environment variable
credentials = service_account.Credentials.from_service_account_file(service_account_path)
# Initializing a BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Defining your dataset and table names
dataset_name = '2ndjsoncmepersonalproject'
fact_table_name = 'trade_fact'
time_dim_table_name = 'time_dim'
security_dim_table_name ='security_dim'

# Creating the dataset
dataset_ref = client.dataset(dataset_name)
client.get_dataset(dataset_ref)

# Defining the schema for the fact table
fact_table_schema = [
    bigquery.SchemaField('securityid', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('transacttime', 'INTEGER'),
    bigquery.SchemaField('instrumentsequencenumber', 'INTEGER'),
    bigquery.SchemaField('recordtype', 'STRING'),
    bigquery.SchemaField('marketsegmentid', 'INTEGER'),
    bigquery.SchemaField('isuserdefinedinstrument', 'STRING')
]

# Defining the schema for the date dimension table
time_dim_table_schema = [
    bigquery.SchemaField('transacttime', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('tradedate', 'DATE'),
    bigquery.SchemaField('nanotime', 'INTEGER')
]

# Defining the schema for the security dimension table
security_dim_table_schema = [
    bigquery.SchemaField('securityid', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('securitytype', 'STRING'),
    bigquery.SchemaField('symbol', 'FLOAT'),
    bigquery.SchemaField('securityexchange', 'STRING'),
    bigquery.SchemaField('currencycode', 'STRING'),
    bigquery.SchemaField('underlyingproduct', 'STRING'),
    bigquery.SchemaField('asset', 'STRING'),
]

# Creating the FACT_TABLE:
fact_table_ref = dataset_ref.table(fact_table_name)
try:
    client.get_table(fact_table_ref)
    print(f"Table {fact_table_name} already exists in the dataset {dataset_name}.")
except:
    fact_table = bigquery.Table(fact_table_ref, schema=fact_table_schema)
    client.create_table(fact_table)
    print(f"{fact_table_name} Created")

# Creating the DIM_TIME table:
time_dim_table_ref = dataset_ref.table(time_dim_table_name)
try:
    client.get_table(time_dim_table_ref)
    print(f"Table {time_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    time_dim_table = bigquery.Table(time_dim_table_ref, schema=time_dim_table_schema)
    client.create_table(time_dim_table)
    print(f"{time_dim_table_name} Created")

# Creating the DIM_Security table:
security_dim_table_ref = dataset_ref.table(security_dim_table_name)
try:
    client.get_table(security_dim_table_ref)
    print(f"Table {security_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    security_dim_table = bigquery.Table(security_dim_table_ref, schema=security_dim_table_schema)
    client.create_table(security_dim_table)
    print(f"{security_dim_table_name} Created")


trade_fact Created
time_dim Created
security_dim Created


# Read a dataset from your Google Cloud Storage into a Pandas DataFrame

In [ ]:
import pandas as pd
from gcsfs import GCSFileSystem

# Replace with you actual bucket and file path
gcs_bucket = 'jsoncme'
gcs_file_path = 'Cleaned_CME_Data.csv'

# Using Pandas to read the dataset from GCS into a DataFrame
df = pd.read_csv(f'gcs://{gcs_bucket}/{gcs_file_path}')

# Displaying the first few rows of the DataFrame
df.head()


,securitytype,symbol,securityexchange,nanotime,transacttime,securityid,currencycode,hour,instrumentsequencenumber,recordtype,marketsegmentid,underlyingproduct,tradedate,asset,isuserdefinedinstrument
0,OPT,UD:WZ: 12 2877604,XCBT,196012091,2023-07-24 02:02:32.196,2877604,USD,2,67,MDV,60,%22NOT_AVAILABLE%22,2023-07-24,OZW,U
1,OPT,UD:WZ: 12 2877604,XCBT,196012091,2023-07-24 02:02:32.196,2877604,USD,2,70,OPNS,60,%22NOT_AVAILABLE%22,2023-07-24,OZW,U
2,OPT,UD:WZ: 12 2877604,XCBT,196012091,2023-07-24 02:02:32.196,2877604,USD,2,71,OPNS,60,%22NOT_AVAILABLE%22,2023-07-24,OZW,U
3,FUT,SR3H7,XCME,354334027,2023-07-24 02:03:39.354,254041,USD,2,78968,MDV,82,Interest Rate,2023-07-24,SR3,H
4,FUT,SR3H7,XCME,356336945,2023-07-24 02:03:39.356,254041,USD,2,78984,MDV,82,Interest Rate,2023-07-24,SR3,H


# Load Data into BigQuery Tables

In [ ]:
# Creating a function that uploads your data to BigQuery from a DataFrame
def upload_data_from_dataframe(df, table_ref):
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.autodetect = True
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

# Splitting your DataFrame into the respective dimension and fact DataFrames
# fact_df, date_dim_df, airport_dim_df, weather_dim_df = split_your_dataframe(df_transformed)
def split_df(df):
    fact_cols = [
    "securityid", "transacttime", "instrumentsequencenumber",
    "recordtype", "marketsegmentid",
    "isuserdefinedinstrument"]

    time_cols = [
    "transacttime", "tradedate", "nanotime"]


    security_cols = [
    "securityid", "securitytype", "symbol", "securityexchange", "currencycode", "underlyingproduct",
    "asset"]


    fact_df = df[fact_cols]
    time_dim_df = df[time_cols]
    security_dim_df = df[security_cols]

    # Returning the split DataFrames
    return fact_df, time_dim_df, security_dim_df

fact_df, time_dim_df, security_dim_df= split_df(df)

# Uploading the data to BigQuery
upload_data_from_dataframe(fact_df, fact_table_ref)
upload_data_from_dataframe(time_dim_df, time_dim_table_ref)
upload_data_from_dataframe(security_dim_df, security_dim_table_ref)